In [1]:
!pip install spacy

In [2]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import spacy
nlp=spacy.load("en_core_web_lg")

In [4]:
!pip install docx2txt
!pip install PyPDF2

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3959 sha256=e10c13d291d26741fdd16c31e4bafd7349aa820a100395acae64fb978b8e2209
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.1 MB/s eta 0:00:00


In [5]:
from flask import Flask, request, render_template
import os
import docx2txt
import PyPDF2
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

resumess=[]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
!pip install PyPDF2

#Extraction Method From Google Cloud#

In [7]:
def similarity(w1, w2):
  w1=nlp(w1)
  w2=nlp(w2)
  return score(w1,w2)



In [8]:
def similarity_nouns(w1_nouns,w2_nouns):
  w1_nouns=nlp(w1_nouns)
  w2_nouns=nlp(w2_nouns)
  similarity_score1=w1_nouns.similarity(w2_nouns)
  return similarity_score1

In [9]:
def similarity_adj(w1_adj, w2_adj):
  w1_adjs=nlp(w1_adj)
  w2_adjs=nlp(w2_adj)
  similarity_score2=w1_adjs.similarity(w2_adjs)
  return similarity_score2

In [10]:
def similarity_verb(w1_verb, w2_verb):
  w1_verbs=nlp(w1_verb)
  w2_verbs=nlp(w2_verb)
  similarity_score3=w1_verbs.similarity(w2_verbs)
  return similarity_score3

In [11]:
def score(w1,w2):
  w1_nouns=" ".join([token.lemma_ for token in w1 if token.pos_=="NOUN"])
  w2_nouns=" ".join([token.lemma_ for token in w2 if token.pos_=="NOUN"])
  w1_adj=" ".join([token.lemma_ for token in w1 if token.pos_=="ADJ"])
  w2_adj=" ".join([token.lemma_ for token in w2 if token.pos_=="ADJ"])
  w1_verb=" ".join([token.lemma_ for token in w1 if token.pos_=="VERB"])
  w2_verb=" ".join([token.lemma_ for token in w2 if token.pos_=="VERB"])
  vector1=similarity_nouns(w1_nouns,w2_nouns)
  vector2=similarity_adj(w1_adj,w2_adj)
  vector3=similarity_verb(w1_verb,w2_verb)
  sum=((vector1 + vector2+ vector3)/3)
  return sum

In [12]:
from google.colab import auth
from google.auth import default
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io
import os
import PyPDF2
import pandas as pd
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

auth.authenticate_user()
creds, _ = default()
drive_service = build('drive', 'v3', credentials=creds)

def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

def cleanUp(text):
    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series([text]).str.replace("[^a-zA-Z]", " ")
    # make alphabets lowercase
    clean_sentences = [s.lower() for s in clean_sentences]
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    return clean_sentences[0]

def getMethod(input: str, path: str):
    resumess = []
    resume_names = []
    try:
        # List files in the specified Google Drive folder
        folder_id = path  # 'path'
        results = drive_service.files().list(
            q=f"'{folder_id}' in parents",
            fields="files(id, name)").execute()
        items = results.get('files', [])

        for item in items:
            file_id = item['id']
            file_name = item['name']

            if file_name.lower().endswith('.pdf'):
                # Download the file content
                request = drive_service.files().get_media(fileId=file_id)
                fh = io.BytesIO()
                downloader = MediaIoBaseDownload(fh, request)
                done = False
                while done is False:
                    status, done = downloader.next_chunk()

                # Read the content
                fh.seek(0)
                pdf_reader = PyPDF2.PdfReader(fh)
                Otext = ""
                for page in pdf_reader.pages:
                    Otext += page.extract_text()

                cleaned_text = cleanUp(Otext)
                resumess.append(cleaned_text)
                resume_names.append(file_name)

                print(f"Processed: {file_name}")

            elif file_name.lower().endswith('.docx'):
                request = drive_service.files().get_media(fileId=file_id)
                fh = io.BytesIO()
                downloader = MediaIoBaseDownload(fh, request)
                done = False
                while done is False:
                    status, done = downloader.next_chunk()

                fh.seek(0)
                text = docx2txt.process(fh)

                cleaned_text = cleanUp(text)
                resumess.append(cleaned_text)
                resume_names.append(file_name)

                print(f"Processed: {file_name}")

            elif file_name.lower().endswith('.txt'):
                request = drive_service.files().get_media(fileId=file_id)
                fh = io.BytesIO()
                downloader = MediaIoBaseDownload(fh, request)
                done = False
                while done is False:
                    status, done = downloader.next_chunk()

                fh.seek(0)
                text = fh.read().decode('utf-8', errors='ignore')

                cleaned_text = cleanUp(text)
                resumess.append(cleaned_text)
                resume_names.append(file_name)

                print(f"Processed: {file_name}")

        print(f"Total PDFs processed: {len(resumess)}")

        vetorizer=TfidfVectorizer().fit_transform([input] + resumess)
        vectors=vetorizer.toarray()
        job_vector=vectors[0]
        resume_vectors=vectors[1:]
        similarities=[]

        for i in range(len(resumess)):
          w=resumess[i]
          similarities.append(similarity(input, w))

        #sort similarities along with indices
        sorted_indices = sorted(range(len(similarities)), key=similarities.__getitem__, reverse=True)
        top_indices=sorted_indices[-5:]
        top_resumes=[resumess[i] for i in top_indices]
        top_resume_names = [resume_names[i] for i in top_indices]
        similarity_scores=[round(similarities[i], 2)*100 for i in top_indices]

        for name, score in zip(top_resume_names, similarity_scores):
          print(f"Resume: {name}, Similarity Score: {score}")

        return resumess


    except Exception as e:
        print(f"An error occurred: {e}")
        return [], []

# processed_resumes = getMethod('job description', '1W76QMo6XSKRLpHbHHLzgDDJuVGWznuVH')
# print(processed_resumes)
job_desc = "Job Description: We are seeking a highly motivated and experienced Software Engineer to join our dynamic team. The ideal candidate will have a strong background in software development, excellent problem-solving skills, and the ability to work effectively in a collaborative environment. Key Responsibilities: - Develop, test, and maintain high-quality software applications. - Collaborate with cross-functional teams to define and design new features. - Troubleshoot and resolve software defects and issues. - Participate in code reviews to ensure adherence to coding standards and best practices. - Continuously learn and apply new technologies to improve software performance and scalability. Requirements: - Bachelor's degree in Computer Science, Engineering, or a related field. - 3+ years of experience in software development. - Proficiency in one or more programming languages (e.g., Python, Java, C++). - Experience with version control systems (e.g., Git). - Strong understanding of software development methodologies (e.g., Agile, Scrum). - Excellent communication and teamwork skills. Preferred Qualifications: - Experience with cloud platforms (e.g., AWS, Azure, Google Cloud). - Familiarity with front-end technologies (e.g., HTML, CSS, JavaScript). - Knowledge of database management systems (e.g., MySQL, PostgreSQL). - Previous experience in a fast-paced startup environment. If you are passionate about software engineering and eager to contribute to the success of a growing company, we encourage you to apply. Please submit your resume and cover letter for consideration. "
outputt = getMethod(job_desc , '1W76QMo6XSKRLpHbHHLzgDDJuVGWznuVH' )
print(outputt)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Processed: software developer.pdf
Processed: backend developer.pdf
Processed: data engineer.pdf
Processed: graphic.docx
Processed: Software Engineer.pdf
Total PDFs processed: 5
Resume: Software Engineer.pdf, Similarity Score: 86.0
Resume: software developer.pdf, Similarity Score: 81.0
Resume: data engineer.pdf, Similarity Score: 81.0
Resume: graphic.docx, Similarity Score: 81.0
Resume: backend developer.pdf, Similarity Score: 79.0
['alice williams alicewilliams@example.com experience: - software developer innovatetech (2019 - present) - developed maintained web applications using javascript, react, node.js. - collaborated ux designers implement user -friendly interfaces. - optimized application performance improved loading times. education: - bachelor science computer science, tech university (2015 - 2019) skills: - programming languages: javascript, python, java - frameworks: react, node.js, express - tools: git, docker, jenkins', 'bob johnson bobjohnson@example.com experience: - back

In [13]:
# Just for testing purpose
similarity("red","pink")

<ipython-input-9-9f733e515850>:4: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_score2=w1_adjs.similarity(w2_adjs)


0.3333333333333333